In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)
itemsUnique=[]
for items in dataset["train"]["BEGIN"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}
print(id2label)

binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["original_response"], truncation=True)
    tempLables=examples["BEGIN"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = binary_dataset.map(preprocess_function,remove_columns=binary_dataset['train'].column_names)
print(dataset)

  0%|          | 0/7 [00:00<?, ?it/s]

{0: 'Hallucination', 1: 'Entailment', 2: 'Uncooperative', 3: 'Generic'}


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

...

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = F"/content/gdrive/My Drive/Models/begin_classification_roberta_base.pt"

In [ ]:
model = torch.load(path)
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
#     warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
#     logging_steps=1000,
    evaluation_strategy='steps',
#     eval_steps=1000,
    save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model='accuracy'
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
#     compute_metrics=load_metric('accuracy')
)

# Fine-tune the model
# trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(encoded_dataset['test'])
print(results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.2807147800922394, 'eval_runtime': 20.109, 'eval_samples_per_second': 175.991, 'eval_steps_per_second': 2.785}


In [ ]:
print(encoded_dataset['test'])

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3539
})


In [ ]:
predictionsOp=trainer.predict(encoded_dataset['test'])
print(predictionsOp)

PredictionOutput(predictions=array([[ 6.3917584 ,  0.18404748, -6.9030695 , -6.8952913 ],
       [-3.9597135 ,  4.1742043 , -0.04775586, -4.4452944 ],
       [-2.7007527 ,  4.295292  , -1.0180308 , -5.4151883 ],
       ...,
       [ 6.317063  ,  0.75067997, -6.999736  , -7.154722  ],
       [ 5.5415106 ,  2.4479766 , -7.0483336 , -7.6445875 ],
       [ 5.229318  ,  4.2027454 , -6.4898705 , -7.152149  ]],
      dtype=float32), label_ids=array([[1., 0., 0., 0.],
       [0., 1., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32), metrics={'test_loss': 0.2807147800922394, 'test_runtime': 18.8366, 'test_samples_per_second': 187.879, 'test_steps_per_second': 2.973})


In [ ]:
for predictionOp in predictionsOp.predictions:
  m = nn.Sigmoid()
  t = torch.from_numpy(predictionOp)
  output = m(t)
  # print(output)

tensor([0.9983, 0.5459, 0.0010, 0.0010])
tensor([0.0187, 0.9848, 0.4881, 0.0116])
tensor([0.0629, 0.9866, 0.2654, 0.0044])
tensor([0.4358, 0.9871, 0.0344, 0.0010])
tensor([9.9486e-01, 9.8565e-01, 1.6462e-03, 8.7957e-04])
tensor([9.9179e-01, 9.8787e-01, 1.6310e-03, 5.9848e-04])
tensor([9.8746e-01, 9.9125e-01, 2.0496e-03, 6.3143e-04])
tensor([9.8241e-01, 9.8890e-01, 2.0341e-03, 4.3276e-04])
tensor([9.9204e-01, 9.8781e-01, 1.5887e-03, 6.0792e-04])
tensor([9.8525e-01, 9.9218e-01, 2.2500e-03, 6.9223e-04])
tensor([0.3120, 0.9923, 0.0464, 0.0011])
tensor([0.1618, 0.9499, 0.1401, 0.0074])
tensor([8.0389e-01, 9.7459e-01, 1.0667e-02, 6.7131e-04])
tensor([0.0237, 0.9863, 0.4003, 0.0086])
tensor([0.0724, 0.9908, 0.1262, 0.0030])
tensor([0.0262, 0.9855, 0.3675, 0.0083])
tensor([6.9839e-01, 9.8537e-01, 1.3691e-02, 6.5316e-04])
tensor([0.3304, 0.9918, 0.0290, 0.0010])
tensor([0.0260, 0.9879, 0.4049, 0.0075])
tensor([9.9038e-01, 9.9116e-01, 2.4579e-03, 9.6330e-04])
tensor([9.7465e-01, 9.8523e-01, 2.27

In [ ]:
prediction_converted=[]
for prediction in predictionsOp.predictions:
  indiarray=[]
  for indi in prediction:
    if indi>1:
      indiarray.append(1.0)
    else:
      indiarray.append(0.0)
  prediction_converted.append(indiarray)
print(prediction_converted)

[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0

In [ ]:
countingDict={}
for i in range (0,len(id2label)):
  countingDict[id2label[i]]=0

In [ ]:
for predictions in prediction_converted:
  # print(predictions)
  for i in range(0,len(predictions)):
    # print(type(predictions[i]))
    if predictions[i]==1.0:
      # print("here")
      countingDict[id2label[i]]+=1

In [ ]:
print(countingDict)

{'Hallucination': 2366, 'Entailment': 2993, 'Uncooperative': 0, 'Generic': 3}


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)
itemsUnique=[]
for items in dataset["train"]["BEGIN"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}
print(id2label)

  0%|          | 0/7 [00:00<?, ?it/s]

{0: 'Hallucination', 1: 'Entailment', 2: 'Uncooperative', 3: 'Generic'}


In [ ]:
binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
testDataset=binary_dataset["test"]
testDataset = testDataset.add_column("Dialogue_output", dialogueModelOutput)

import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["Dialogue_output"], truncation=True,padding='max_length')
    tempLables=examples["BEGIN"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = testDataset.map(preprocess_function,remove_columns=testDataset.column_names)
print(dataset)

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

  0%|          | 0/3539 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [ ]:
print(encoded_dataset)
print(len(encoded_dataset['input_ids'][0]))
print(len(encoded_dataset['attention_mask'][0]))
print(len(encoded_dataset['labels'][0]))

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3539
})
512
512
4


In [ ]:
predictionsOp=trainer.predict(encoded_dataset)
print(predictionsOp)